In [9]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

data = pd.read_csv("/media/leon/DISK2/icig/done/paper/2025.10.31_input_for_py_dp50_frac.csv")
results = []

for _, row in data.iterrows():
    gene = row["symbol"]
    patients = row["patients"]

    # Build long format directly from DP columns
    df_long = pd.DataFrame({
        "patient": [patients.split("_")[0], patients.split("_")[1],
                    patients.split("_")[0], patients.split("_")[1]],
        "condition": ["0", "0", "1", "1"],
        "maj": [row["DP_maj_gene_0_p1"], row["DP_maj_gene_0_p2"],
                row["DP_maj_gene_1_p1"], row["DP_maj_gene_1_p2"]],
        "min": [row["DP_min_gene_0_p1"], row["DP_min_gene_0_p2"],
                row["DP_min_gene_1_p1"], row["DP_min_gene_1_p2"]],
    })
    df_long["total"] = df_long["maj"] + df_long["min"]

    # Melt to long format for logistic regression
    df_long = pd.melt(df_long,
                      id_vars=["patient", "condition", "total"],
                      value_vars=["maj", "min"],
                      var_name="allele_type",
                      value_name="count")
    df_long["is_maj"] = (df_long["allele_type"] == "maj").astype(int)

    # df_long['freq'] = df_long['count'] / df_long['total']

    # Fit logistic regression with interaction
    model = smf.glm("is_maj ~ condition * patient",
                    data=df_long,
                    family=sm.families.Binomial(),
                    freq_weights=df_long["count"])
    result = model.fit()

    # Extract p-value for interaction
    interaction_terms = [t for t in result.pvalues.index if ":" in t]
    if interaction_terms:
        p_interaction = result.pvalues[interaction_terms[-1]]
        coef_interaction = result.params[interaction_terms[-1]]
    else:
        p_interaction = None
        coef_interaction = None

    results.append({
        "symbol": gene,
        "patients": patients,
        "p_interaction": p_interaction,
        "coef_interaction": coef_interaction
    })

res_df = pd.DataFrame(results)
res_df = res_df.dropna(subset=["p_interaction"])
res_df
# res_df.to_csv("/media/leon/DISK2/icig/done/paper/load_to_r.csv", index=False)

,symbol,patients,p_interaction,coef_interaction
0,FAAP20,sAn_sBn,0.671566,-0.086360
1,DFFB,sAn_sDm,0.878947,0.065586
2,C1orf174,sAn_sBn,0.185070,-0.394593
3,PHF13,sAn_sBn,0.829894,-0.075434
4,ENO1,sAn_sBn,0.050060,-1.025290
...,...,...,...,...
3659,PSMB7,sBn_sDm,0.401657,0.075746
3660,URM1,sBn_sDm,0.003726,0.894244
3661,FAM78A,sBn_sDm,0.522252,0.113584
3662,ENTR1,sBn_sDm,0.208234,-0.397473


In [10]:
# Swap before and after columns for the second patient if there was no common SNPs
data = data[data.frac_common < 0.5]
data["DP_maj_gene_0_p2"], data["DP_maj_gene_1_p2"] = data["DP_maj_gene_1_p2"], data["DP_maj_gene_0_p2"]
data["DP_min_gene_0_p2"], data["DP_min_gene_1_p2"] = data["DP_min_gene_1_p2"], data["DP_min_gene_0_p2"]
data

,symbol,patients,DP_maj_gene_0_p1,DP_min_gene_0_p1,DP_maj_gene_1_p1,DP_min_gene_1_p1,DP_maj_gene_0_p2,DP_min_gene_0_p2,DP_maj_gene_1_p2,DP_min_gene_1_p2,frac_common
0,FAAP20,sAn_sBn,275,260,344,426,123,146,160,133,0.00
1,DFFB,sAn_sDm,67,38,36,36,33,38,76,53,0.00
2,C1orf174,sAn_sBn,72,57,112,66,123,129,119,113,0.00
3,PHF13,sAn_sBn,60,59,122,89,63,42,72,60,0.00
4,ENO1,sAn_sBn,42,26,41,52,18,35,47,16,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1955,SLX4,sAn_sDm,71,49,59,65,53,56,46,39,0.47
1956,NASP,sBn_sDm,112,72,86,69,195,185,188,166,0.47
1957,YIPF4,sAn_sDm,117,35,98,35,85,28,100,28,0.48
1958,GNB4,sBn_sDm,85,42,90,49,107,61,87,50,0.48


In [11]:
results = []

for _, row in data.iterrows():
    gene = row["symbol"]
    patients = row["patients"]

    # Build long format directly from DP columns
    df_long = pd.DataFrame({
        "patient": [patients.split("_")[0], patients.split("_")[1],
                    patients.split("_")[0], patients.split("_")[1]],
        "condition": ["0", "0", "1", "1"],
        "maj": [row["DP_maj_gene_0_p1"], row["DP_maj_gene_0_p2"],
                row["DP_maj_gene_1_p1"], row["DP_maj_gene_1_p2"]],
        "min": [row["DP_min_gene_0_p1"], row["DP_min_gene_0_p2"],
                row["DP_min_gene_1_p1"], row["DP_min_gene_1_p2"]],
    })
    df_long["total"] = df_long["maj"] + df_long["min"]

    # Melt to long format for logistic regression
    df_long = pd.melt(df_long,
                      id_vars=["patient", "condition", "total"],
                      value_vars=["maj", "min"],
                      var_name="allele_type",
                      value_name="count")
    df_long["is_maj"] = (df_long["allele_type"] == "maj").astype(int)

    # df_long['freq'] = df_long['count'] / df_long['total']

    # Fit logistic regression with interaction
    model = smf.glm("is_maj ~ condition * patient",
                    data=df_long,
                    family=sm.families.Binomial(),
                    freq_weights=df_long["count"])
    result = model.fit()

    # Extract p-value for interaction
    interaction_terms = [t for t in result.pvalues.index if ":" in t]
    if interaction_terms:
        p_interaction = result.pvalues[interaction_terms[-1]]
        coef_interaction = result.params[interaction_terms[-1]]
    else:
        p_interaction = None
        coef_interaction = None

    results.append({
        "symbol": gene,
        "patients": patients,
        "p_interaction": p_interaction,
        "coef_interaction": coef_interaction
    })

res_df2 = pd.DataFrame(results)
res_df2 = res_df2.dropna(subset=["p_interaction"])
res_df2

,symbol,patients,p_interaction,coef_interaction
0,FAAP20,sAn_sBn,0.002111,0.626134
1,DFFB,sAn_sDm,0.013081,1.068626
2,C1orf174,sAn_sBn,0.510637,-0.195866
3,PHF13,sAn_sBn,0.137314,-0.521721
4,ENO1,sAn_sBn,0.000003,2.459780
...,...,...,...,...
1955,SLX4,sAn_sDm,0.076639,0.687849
1956,NASP,sBn_sDm,0.270220,0.293402
1957,YIPF4,sAn_sDm,0.408704,0.339725
1958,GNB4,sBn_sDm,0.800894,0.088922


In [15]:
merged_df = pd.concat([res_df, res_df2], ignore_index=True)
merged_df = merged_df.sort_values('p_interaction', ascending=False).drop_duplicates(['symbol', 'patients'], keep='first')
merged_df
# merged_df.to_csv("/media/leon/DISK2/icig/done/paper/2025.10.31_glm_output_dp50_frac.csv", index=False)

,symbol,patients,p_interaction,coef_interaction
1636,CD6,sBn_sDm,9.999936e-01,-9.542907e-07
531,LITAF,sBn_sDm,9.999491e-01,-5.364678e-06
4679,POLR2D,sBn_sDm,9.999075e-01,4.542254e-05
450,VSIR,sBn_sDm,9.997447e-01,-4.285096e-05
5034,ABCF2-H2BK1,sBn_sDm,9.993873e-01,-1.690188e-04
...,...,...,...,...
2395,LILRB2,sAn_sDm,5.895617e-08,-5.244696e-01
2069,MYO1C,sAn_sBn,2.384964e-08,1.553812e+00
2000,TP53I11,sAn_sBn,1.106750e-08,1.448096e+00
3719,TP53I11,sAn_sDm,3.253661e-10,1.453061e+00


In [17]:
sum(merged.p_interaction) / len(merged.p_interaction)

0.49380211328837464

In [14]:
def run_glm(data):
    results = []
    for _, r in data.iterrows():
        p1, p2 = r["patients"].split("_")
        df = pd.DataFrame({
            "patient": [p1, p2, p1, p2],
            "condition": ["0", "0", "1", "1"],
            "maj": [r[f"DP_maj_gene_{i}_p{j}"] for i in (0, 1) for j in (1, 2)],
            "min": [r[f"DP_min_gene_{i}_p{j}"] for i in (0, 1) for j in (1, 2)],
        })
        df["total"] = df["maj"] + df["min"]
        df = df.melt(id_vars=["patient", "condition", "total"],
                     value_vars=["maj", "min"],
                     var_name="allele_type", value_name="count")
        df["is_maj"] = (df["allele_type"] == "maj").astype(int)

        model = smf.glm("is_maj ~ condition * patient",
                        data=df, family=sm.families.Binomial(),
                        freq_weights=df["count"]).fit()

        inter = [t for t in model.pvalues.index if ":" in t]
        if inter:
            results.append({
                "symbol": r["symbol"], "patients": r["patients"],
                "p_interaction": model.pvalues[inter[-1]],
                "coef_interaction": model.params[inter[-1]]
            })
    return pd.DataFrame(results).dropna(subset=["p_interaction"])


# ---- main ----
data = pd.read_csv("/media/leon/DISK2/icig/done/paper/2025.10.31_input_for_py_dp50_frac.csv")

res_df1 = run_glm(data)

# Swap columns for n_common == 0 or frac_common < 0.5
if "n_common" in data.columns:
    swapped = data[data["n_common"] == 0].copy()
elif "frac_common" in data.columns:
    swapped = data[data["frac_common"] < 0.5].copy()
for c in ["DP_maj_gene", "DP_min_gene"]:
    swapped[[f"{c}_0_p2", f"{c}_1_p2"]] = swapped[[f"{c}_1_p2", f"{c}_0_p2"]].values

res_df2 = run_glm(swapped)

merged = pd.concat([res_df1, res_df2], ignore_index=True)
merged = merged.sort_values("p_interaction", ascending=False).drop_duplicates(["symbol", "patients"])
merged#.to_csv(output_path, index=False)

,symbol,patients,p_interaction,coef_interaction
1636,CD6,sBn_sDm,9.999936e-01,-9.542907e-07
531,LITAF,sBn_sDm,9.999491e-01,-5.364678e-06
4679,POLR2D,sBn_sDm,9.999075e-01,4.542254e-05
450,VSIR,sBn_sDm,9.997447e-01,-4.285096e-05
5034,ABCF2-H2BK1,sBn_sDm,9.993873e-01,-1.690188e-04
...,...,...,...,...
2395,LILRB2,sAn_sDm,5.895617e-08,-5.244696e-01
2069,MYO1C,sAn_sBn,2.384964e-08,1.553812e+00
2000,TP53I11,sAn_sBn,1.106750e-08,1.448096e+00
3719,TP53I11,sAn_sDm,3.253661e-10,1.453061e+00


In [8]:
len(set([i[1][0] for i in res_df.iterrows() if i[1][2] < 0.01]))

/tmp/ipykernel_2727587/886012866.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  len(set([i[1][0] for i in res_df.iterrows() if i[1][2] < 0.01]))


2022

In [48]:
min(res_df.p_interaction)

0.5605925448078448